In [ ]:
%reload_ext autoreload
%autoreload 2
import networkqit as nq
import numpy as np
import scipy as sp
from numpy.random import random as rand
from networkqit import graph_laplacian as GL
from scipy.linalg import eigvalsh
from scipy.special import logsumexp
import numdifftools as nd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
sns.set()
sns.set_palette('Blues')

In [ ]:
def ER(n,p):
    T = np.triu(rand([n,n]),1)
    T = (T + T.T)
    A = (T <= p).astype(float)
    np.fill_diagonal(A,0)
    return A

def LER(n,p):
    return GL(ER(n,p))

In [ ]:
f = lambda n,beta,p : logsumexp(-beta*eigvalsh(LER(n,p)))

In [ ]:
plt.figure(figsize = (12,8))
p = np.linspace(0,1,100)
n = 100
for beta in np.logspace(-2,2,10):
    plt.plot([nd.Derivative(lambda x : f(n,beta,x))(pi) for pi in p])
plt.show()

In [ ]:
N=10
A=ER(N,0.8)
L=GL(A)

M=nq.ErdosRenyi(N=N)
x0=np.random.random(1,)
print('Density=',A.sum()/(N*(N-1)))

solver = nq.Adam(A=A,x0=x0,beta_range=np.logspace(0,-3,5))
print('Starting from',solver.x0)
solver.setup(expected_adj_fun = M, adj_sampling_fun = lambda x : ER(N,x), expected_laplacian_grad=M.expected_laplacian_grad, step_callback = lambda beta,p : print('\rbeta=',beta,'p=',p[0],end=''))
sol,all_x = solver.run(max_iters=2000,alpha=1E-1,num_samples=1000)

In [ ]:
beta=1
def replicatrick(n,p):
    eps = 1E-9
    l = eigvalsh(LER(n,p))
    x = logsumexp(-beta*l)
    y = (np.sum(np.exp(-beta*l))**eps-1)/eps
    print(x-y)

def quenched_free_energy(n,p,beta,reps): # the correct calculation
    fannealed = 0
    for r in range(0,reps):
        l = eigvalsh(LER(n,p))
        fannealed += logsumexp(-beta*l)/reps
    return fannealed

def annealed_free_energy(n,p,beta,reps):
    fannealed = 0
    for r in range(0,reps):
        l = eigvalsh(LER(n,p))
        fannealed += (np.exp(-beta*l)).sum()
    return np.log(fannealed/reps)

beta_range = np.logspace(-5,5,100)
p=0.1
n=50
plt.figure(figsize=(16,8))
plt.semilogx(1/beta_range,[quenched_free_energy(n,p,beta,200) for beta in beta_range],'b')
plt.semilogx(1/beta_range,[annealed_free_energy(n,p,beta,200) for beta in beta_range],'r')
plt.legend(['Quenched','Annealed'])
plt.xlabel('$1/\\beta$')
plt.show()

In [ ]:
beta_range = np.logspace(-5,5,100)
n=10
reps=10000
beta=0.01
dquenched_free_energy = nd.Derivative(lambda p: quenched_free_energy(n,p,beta,reps))
allp=np.linspace(0,1,20)
plt.figure(figsize=(16,8))
plt.semilogx(allp,[(quenched_free_energy(n,p+1E-4,beta,reps)-quenched_free_energy(n,p,beta,reps))/1E-4 for p in allp],'bo')
plt.semilogx(allp,[dquenched_free_energy(p) for p in allp ],'b-')
#plt.semilogx(1/beta_range,[quenched_free_energy(n,p+1E-1,beta,20) for beta in beta_range],'r.-')
plt.legend(['limit','precise'])
plt.xlabel('$p$')
plt.show()